In [1]:
import numpy as np
from rl.processors import WhiteningNormalizerProcessor
from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess

In [2]:
import random
from keras.models import Sequential, model_from_json, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
import tensorflow as tf
import json
import pandas as pd
import keras

In [3]:
import math
from keras.initializers import normal, identity
from keras.models import model_from_json
from keras.models import Sequential, Model
#from keras.engine.training import collect_trainable_weights
from keras.layers import Dense, Flatten, Input, merge, Lambda
from keras.optimizers import Adam
import tensorflow as tf
import keras.backend as K
from tqdm.auto import tqdm

In [4]:
#Ornstein-Uhlenbeck Process
class OU(object):

    def function(self, x, mu, theta, sigma):
        return theta * (mu - x) + sigma * np.random.randn(1)

In [5]:
OU = OU()

In [6]:
BUFFER_SIZE = 100000
BATCH_SIZE = 32
GAMMA = 0.99
TAU = 0.001     #Target Network HyperParameters
LRA = 0.0001    #Learning rate for Actor
LRC = 0.001     #Lerning rate for Critic

In [7]:
action_dim = 4  #Przeplyw powietrza / Zawartosc tlenu / Predkosc Dmuchu / Nadawa pylow
                #'001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv'
state_dim = 22  #parametry wyjsciowe
np.random.seed(234)

In [8]:
EXPLORE = 100000.
episode_count = 2000
max_steps = 100000
reward = 0
done = False
step = 0
epsilon = 1
indicator = 0

In [338]:
config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
from keras import backend as K
K.set_session(sess)

In [9]:
sess = keras.backend.get_session()
init = tf.compat.v1.global_variables_initializer()
K.set_session(sess)

In [22]:
# sternik
tf.compat.v1.disable_eager_execution()

HIDDEN1_UNITS = 300
HIDDEN2_UNITS = 600

class ActorNetwork(object):
    def __init__(self, sess, state_size, action_size, BATCH_SIZE, TAU, LEARNING_RATE):
        self.sess = sess
        self.BATCH_SIZE = BATCH_SIZE
        self.TAU = TAU
        self.LEARNING_RATE = LEARNING_RATE

        K.set_session(sess)

        #Now create the model
        self.model , self.weights, self.state = self.create_actor_network(state_size, action_size)   
        self.target_model, self.target_weights, self.target_state = self.create_actor_network(state_size, action_size) 
        self.action_gradient = tf.compat.v1.placeholder(tf.float32,[None, action_size])
        self.params_grad = tf.gradients(self.model.output, self.weights, -self.action_gradient)
        grads = zip(self.params_grad, self.weights)
        self.optimize = tf.optimizers.Adam(LEARNING_RATE).apply_gradients(grads)
        self.sess.run(tf.compat.v1.initialize_all_variables())

    def train(self, states, action_grads):
        self.sess.run(self.optimize, feed_dict={
            self.state: states,
            self.action_gradient: action_grads
        })

    def target_train(self):
        actor_weights = self.model.get_weights()
        actor_target_weights = self.target_model.get_weights()
        for i in xrange(len(actor_weights)):
            actor_target_weights[i] = self.TAU * actor_weights[i] + (1 - self.TAU)* actor_target_weights[i]
        self.target_model.set_weights(actor_target_weights)

    def create_actor_network(self, state_size,action_dim):
        S = Input(shape=[state_size])   
        h0 = Dense(HIDDEN1_UNITS, activation='relu')(S)
        h1 = Dense(HIDDEN2_UNITS, activation='relu')(h0)
        Przeplyw = Dense(1,activation='sigmoid')(h1)  
        Predkosc = Dense(1,activation='sigmoid')(h1)   
        Tlen = Dense(1,activation='sigmoid')(h1) 
        Pyly = Dense(1,activation='relu')(h1) 
        V = merge.concatenate([Przeplyw,Predkosc,Tlen,Pyly], axis=-1)        
        model = Model(inputs=S,outputs=V)
        return model, model.trainable_weights, S

In [14]:
actor = ActorNetwork(sess, state_dim, action_dim, BATCH_SIZE, TAU, LRA)

In [15]:
class CriticNetwork(object):
    def __init__(self, sess, state_size, action_size, BATCH_SIZE, TAU, LEARNING_RATE):
        self.sess = sess
        self.BATCH_SIZE = BATCH_SIZE
        self.TAU = TAU
        self.LEARNING_RATE = LEARNING_RATE
        self.action_size = action_size
        
        K.set_session(sess)

        #Now create the model
        self.model, self.action, self.state = self.create_critic_network(state_size, action_size)  
        self.target_model, self.target_action, self.target_state = self.create_critic_network(state_size, action_size)  
        self.action_grads = tf.gradients(self.model.output, self.action)  #GRADIENTS for policy update
        self.sess.run(tf.compat.v1.initialize_all_variables())

    def gradients(self, states, actions):
        return self.sess.run(self.action_grads, feed_dict={
            self.state: states,
            self.action: actions
        })[0]

    def target_train(self):
        critic_weights = self.model.get_weights()
        critic_target_weights = self.target_model.get_weights()
        for i in xrange(len(critic_weights)):
            critic_target_weights[i] = self.TAU * critic_weights[i] + (1 - self.TAU)* critic_target_weights[i]
        self.target_model.set_weights(critic_target_weights)

    def create_critic_network(self, state_size,action_dim):
        S = Input(shape=[state_size])  
        A = Input(shape=[action_dim],name='action2')   
        w1 = Dense(HIDDEN1_UNITS, activation='relu')(S)
        a1 = Dense(HIDDEN2_UNITS, activation='linear')(A) 
        h1 = Dense(HIDDEN2_UNITS, activation='linear')(w1)
        h2 = merge.add([h1,a1])    
        h3 = Dense(HIDDEN2_UNITS, activation='relu')(h2)
        V = Dense(action_dim,activation='linear')(h3)   
        model = Model(inputs=[S,A],outputs=V)
        #adam = Adam(lr=self.LEARNING_RATE)
        model.compile(loss='mse', optimizer='adam')
        return model, A, S 

In [16]:
critic = CriticNetwork(sess, state_dim, action_dim, BATCH_SIZE, TAU, LRC)

Now we build the model
Now we build the model
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [17]:
from collections import deque
import random

class ReplayBuffer(object):

    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.num_experiences = 0
        self.buffer = deque()

    def getBatch(self, batch_size):
        # Randomly sample batch_size examples
        if self.num_experiences < batch_size:
            return random.sample(self.buffer, self.num_experiences)
        else:
            return random.sample(self.buffer, batch_size)

    def size(self):
        return self.buffer_size

    def add(self, state, action, reward, new_state):
        experience = (state, action, reward, new_state)
        if self.num_experiences < self.buffer_size:
            self.buffer.append(experience)
            self.num_experiences += 1
        else:
            self.buffer.popleft()
            self.buffer.append(experience)

    def count(self):
        # if buffer is full, return buffer size
        # otherwise, return experience counter
        return self.num_experiences

    def erase(self):
        self.buffer = deque()
        self.num_experiences = 0

In [18]:
buff = ReplayBuffer(BUFFER_SIZE)

In [19]:
def split_seq(data_x, data_y, steps):
    x, y = list(), list()
    for i in range(len(data_x) - steps):
        end = i + steps
        seq_x = data_x[i:end, :]
        seq_y = data_y[end, :]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

In [65]:
data = pd.read_csv('../../../zadanie2_7z/hmg/joined/data_2021-04-19.csv').drop(columns = ['Unnamed: 0'])
data_y = data.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
data_x = data.to_numpy()

In [20]:
class Furnance:
    terminal_judge_start = 100  # If after 100 timestep still no progress, terminated
    termination_limit_progress = 5  # [km/h], episode terminates if car is running slower than this limit
    default_speed = 50

    initial_reset = True

    def __init__(self, throttle=False, gear_change=False):
        
        self.time_step = 0
        data = pd.read_csv('../../../zadanie2_7z/hmg/joined/data_2021-04-19.csv').drop(columns = ['Unnamed: 0'])
        data_y = data.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
        data_x = data.to_numpy()
        x, y = split_seq(data_x, data_y, steps = 5)
        self.x = x[0].reshape(1,5,26)
        self.y = y[0]
        self.model = None
        self.model = keras.models.load_model('../../../zadanie2_7z/hmg/results/RL/model_last4.h5')
        print("model loaded")

    def model_pred(self, input_x = None):
        
        if input_x is None:
            input_x = self.initial_state
        y_pred = self.model.predict(input_x, verbose=0)
        
        return(y_pred)
        
    def step(self, x):

        y = self.model_pred(x)
        
        heat_loss = abs(y[0][0] - self.y[0]) * -1
        settings = x[0, 4, :4]
        
        reward = heat_loss
        
        if settings[0] < 1900 or settings[0] > 3500:
            reward -= 4 # przeplyw 
        if settings[1] < 65 or settings[1] > 81:
            reward -= 4 # tlen
        if settings[2] < 40 or settings[2] > 70:
            reward -= 4 # predkosc
        if settings[3] < 13:
            reward -= 6 # predkosc
        elif settings[3] < 22:
            reward -= 1
        elif settings[3] > 27:
            reward -= 6
        
        self.y = y

        self.time_step += 1

        return self.y, reward

In [23]:
actor = ActorNetwork(sess, state_dim, action_dim, BATCH_SIZE, TAU, LRA)
critic = CriticNetwork(sess, state_dim, action_dim, BATCH_SIZE, TAU, LRC)
buff = ReplayBuffer(BUFFER_SIZE)
sess.run(tf.compat.v1.initialize_all_variables())
K.set_session(sess)
train_indicator = 1

ob = Furnance()
#initial y
s_t = ob.y

for i in tqdm(range(episode_count)):

        print("Episode : " + str(i) + " Replay Buffer " + str(buff.count()))
     
        total_reward = 0.
        for j in range(max_steps):
            loss = 0 
            epsilon -= 1.0 / EXPLORE
            a_t = np.zeros([1, action_dim])
            noise_t = np.zeros([1, action_dim])
            
            a_t_original = actor.model.predict(s_t.reshape(1, s_t.shape[0]))
            noise_t[0][0] = train_indicator * max(epsilon, 0) * OU.function(a_t_original[0][0],  2700 , 60, 80)
            noise_t[0][1] = train_indicator * max(epsilon, 0) * OU.function(a_t_original[0][1],  70 , 0.013, 0.013)
            noise_t[0][2] = train_indicator * max(epsilon, 0) * OU.function(a_t_original[0][2], 50 , 2, 2)
            noise_t[0][3] = train_indicator * max(epsilon, 0) * OU.function(a_t_original[0][2], 25 , 12/300, 12)

            a_t[0][0] = a_t_original[0][0] + noise_t[0][0]
            a_t[0][1] = a_t_original[0][1] + noise_t[0][1]
            a_t[0][2] = a_t_original[0][2] + noise_t[0][2]
            a_t[0][3] = a_t_original[0][3] + noise_t[0][3]

            nx = ob.x[:, 1:, :]
            new = np.concatenate((a_t[0], s_t)).reshape(1, 1, 26)
            nx = np.concatenate((nx, new), axis = 1)
            
            #r_t = reward
            s_t1, r_t = ob.step(nx)
        
            buff.add(s_t, a_t[0], r_t, s_t1)      #Add replay buffer
            
            #Do the batch update
            batch = buff.getBatch(BATCH_SIZE)
            states = np.asarray([e[0] for e in batch])
            actions = np.asarray([e[1] for e in batch])
            rewards = np.asarray([e[2] for e in batch])
            new_states = np.asarray([e[3] for e in batch])
            y_t = np.asarray([e[1] for e in batch])

            target_q_values = critic.target_model.predict([new_states[0], actor.target_model.predict(new_states[0])])  
           
            for k in range(len(batch)):
                y_t[k] = rewards[k] + GAMMA*target_q_values[k]
       
            if (train_indicator):
                loss += critic.model.train_on_batch([states,actions], y_t) 
                a_for_grad = actor.model.predict(states)
                grads = critic.gradients(states, a_for_grad)
                actor.train(states, grads)
                actor.target_train()
                critic.target_train()

            total_reward += r_t
            s_t = s_t1
        
            print("Episode", i, "Step", step, "Action", a_t, "Reward", r_t, "Loss", loss)
        
            step += 1

        if np.mod(i, 3) == 0:
            if (train_indicator):
                print("Now we save model")
                actor.model.save("../../../zadanie2_7z/hmg/results/RL/actormodel.h5", overwrite=True)
                critic.model.save("../../../zadanie2_7z/hmg/results/RL/criticmodel.h5", overwrite=True)

        print("TOTAL REWARD @ " + str(i) +"-th Episode  : Reward " + str(total_reward))
        print("Total Step: " + str(step))
        print("")

print("Finish.")

Now we build the model
Now we build the model
model loaded


Episode : 0 Replay Buffer 0



FailedPreconditionError: Could not find variable training_2/Adam/iter. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status=Not found: Resource localhost/training_2/Adam/iter/N10tensorflow3VarE does not exist.
	 [[{{node training_2/Adam/ReadVariableOp}}]]